---
# MLlib Spark - Clusterização

## Aprendizado de Máquina no Apache Spark

Alunos:


*   **Alisson Nunes** (725862)
*   **Lucas Mathaeus** (726561)
*   **William Eugênio** (726601)

Bacharelado em Ciência da Computação (BCC)<br>
Departamento de Computação (DC)<br>
Universidade Federal de São Carlos (UFSCar)


# (0) Contextualização

## Aprendizado de Máquina

- .
- .

## Clusterização

## Resilient Distributed Dataset (RDD)

#  (1) Instalação e Configuração

In [ ]:
! pip install -q pyspark numpy

In [ ]:
# Configurando as variaveis de ambiente
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [ ]:
# Definindo uma sessão e contexto Spark
from pyspark.sql import SparkSession
ss = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()
sc = ss.sparkContext

22/08/25 23:03:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/25 23:03:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# (2) Obtenção dos Dados

In [ ]:
# Instalando o módulo wget
%%capture
! pip install -q wget
!rm -rf data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
# Baixando os dados das tabelas de dimensão do data mart Exame
import wget

url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/kmeans_data.txt"
wget.download(url, "data/kmeans_data.txt")

'data/sample_binary_classification_data (1).txt'

In [ ]:
# Carregando o arquivo de dados
from numpy import array

data = sc.textFile("data/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# (3) Tratamento dos Dados

In [ ]:
from pyspark.mllib.clustering import KMeans

clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

PythonRDD[8] at RDD at PythonRDD.scala:53

In [ ]:
from math import sqrt

def error(point):
	center = clusters.centers[clusters.predict(point)]
	return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))


22/08/25 22:38:28 WARN Instrumentation: [8bfb2d8c] Initial coefficients will be ignored! Its dimensions (1, 692) did not match the expected size (1, 692)
22/08/25 22:38:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/08/25 22:38:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [ ]:
from pyspark.mllib.clustering import KMeansModel

clusters.save(sc, "data/KMeansModel")
sameModel = KMeansModel.load(sc, "data/KMeansModel")

In [ ]:
sc.stop()